In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None, 'display.max_rows', None)

In [ ]:
df_pbp = pd.read_csv('nba_pbp_2016-2019.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (71,89,90) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df_id = df_pbp[['PLAYER1_ID', 'PLAYER1_NAME']]

In [ ]:
df_id = df_id.drop_duplicates()
df_id = df_id.dropna()

In [ ]:
print(df_id[df_id['PLAYER1_ID'] == '1626172'])

      PLAYER1_ID  PLAYER1_NAME
12289    1626172  Kevon Looney


In [ ]:
df_id.to_csv('df_index.csv', index=False)

In [ ]:
df_pbp = df_pbp.drop(columns=['Unnamed: 0','EVENTMSGACTIONTYPE','PERSON1TYPE', 'PERSON2TYPE', 'PERSON3TYPE', 'HOMEDESCRIPTION', 'PERIOD', 'VISITORDESCRIPTION', 'EVENTMSGTYPE', 'EVENTNUM', 'NEUTRALDESCRIPTION', 'PCTIMESTRING', 'PLAYER1_TEAM_CITY','PLAYER1_TEAM_NICKNAME', 'PLAYER2_TEAM_CITY', 'PLAYER2_TEAM_NICKNAME', 'PLAYER3_TEAM_CITY', 'PLAYER3_TEAM_NICKNAME','SCORE','SCOREMARGIN', 'WCTIMESTRING', 'TEAM', 'TYPE', 'SUB_TYPE','JUMP_BALL_HOME_PLAYER_ID', 'JUMP_BALL_AWAY_PLAYER_ID', 'JUMP_BALL_RETRIEVED_PLAYER_ID', 'SUB_ENTERED_PLAYER_ID',  'SUB_LEAVING_PLAYER_ID', 'TEAM_FOUL_COUNT', 'FOUL_TYPE', 'TECHNICAL_PLAYER_ID', 'DOUBLE_TECHNICAL_PLAYER_ID_1', 'DOUBLE_TECHNICAL_PLAYER_ID_2', 'DOUBLE_PERSONAL_PLAYER_ID_1', 'DOUBLE_PERSONAL_PLAYER_ID_2', 'DOUBLE_PERSONAL_FOUL_COUNT_1', 'DOUBLE_PERSONAL_FOUL_COUNT_2', 'COACH_TECHNICAL', 'POINTS_SCORED', 'SHOT_TYPE', 'TEAM_TURNOVER_COUNT', 'TURNOVER_TYPE', 'POSSESSION_ID', 'PERIOD_START', 'PERIOD_END', 'TIMEOUT_TYPE', 'VIOLATION', 'VIOLATION_PLAYER_ID', 'EJECTION', 'EJECTION_PLAYER_ID'])

In [ ]:
df_pbp[['PLAYER1_ID', 'HOME_PLAYER_ID_1', 'HOME_PLAYER_ID_2', 'HOME_PLAYER_ID_3', 'HOME_PLAYER_ID_4', 'HOME_PLAYER_ID_5', 'AWAY_PLAYER_ID_1', 'AWAY_PLAYER_ID_2', 'AWAY_PLAYER_ID_3', 'AWAY_PLAYER_ID_4', 'AWAY_PLAYER_ID_5']] = \
 df_pbp[['PLAYER1_ID', 'HOME_PLAYER_ID_1', 'HOME_PLAYER_ID_2', 'HOME_PLAYER_ID_3', 'HOME_PLAYER_ID_4', 'HOME_PLAYER_ID_5', 'AWAY_PLAYER_ID_1', 'AWAY_PLAYER_ID_2', 'AWAY_PLAYER_ID_3', 'AWAY_PLAYER_ID_4', 'AWAY_PLAYER_ID_5']].astype(str)


In [ ]:
df_pbp['lineup_h'] = df_pbp.apply(lambda x:"_".join(sorted(x[['HOME_PLAYER_ID_1', 'HOME_PLAYER_ID_2', 'HOME_PLAYER_ID_3', 'HOME_PLAYER_ID_4', 'HOME_PLAYER_ID_5']].astype(str).to_list())) , axis=1)

In [ ]:
df_pbp['lineup_a'] = df_pbp.apply(lambda x:"_".join(sorted(x[['AWAY_PLAYER_ID_1', 'AWAY_PLAYER_ID_2', 'AWAY_PLAYER_ID_3', 'AWAY_PLAYER_ID_4', 'AWAY_PLAYER_ID_5']].astype(str).to_list())) , axis=1)

In [ ]:
df_pbp['prev_time'] = df_pbp['TIME'].shift(1)
df_pbp['prev_time'] = df_pbp['prev_time'].fillna(0)

In [ ]:
df_pbp['time_per_action'] = df_pbp['TIME'] - df_pbp['prev_time']
df_pbp['time_per_action'] = np.where(df_pbp['time_per_action'] < 0, 0, df_pbp['time_per_action'])

In [ ]:
def statsCounter(stat_name, new_column_name, if_team_stat, if_home):
  
  if (if_home == True):
    df_pbp[stat_name] = df_pbp[stat_name].fillna(0)
    df_pbp[stat_name] = df_pbp[stat_name].astype(str)
    df_pbp[stat_name] = df_pbp[stat_name].apply(lambda x: x[:-2])
    if (if_team_stat == True):
      df_pbp[new_column_name] = np.where((df_pbp[stat_name] == df_pbp['HOME_PLAYER_ID_5'])
                                                  |(df_pbp[stat_name] == df_pbp['HOME_PLAYER_ID_4'])
                                                  |(df_pbp[stat_name] == df_pbp['HOME_PLAYER_ID_3'])
                                                  |(df_pbp[stat_name] == df_pbp['HOME_PLAYER_ID_2'])
                                                  |(df_pbp[stat_name] == df_pbp['HOME_PLAYER_ID_1'])
                                                  |(df_pbp[stat_name] == df_pbp['PLAYER1_ID'])
                                                  , 1, 0)
    else:
      df_pbp[new_column_name] = np.where((df_pbp[stat_name] == df_pbp['HOME_PLAYER_ID_5'])
                                                  |(df_pbp[stat_name] == df_pbp['HOME_PLAYER_ID_4'])
                                                  |(df_pbp[stat_name] == df_pbp['HOME_PLAYER_ID_3'])
                                                  |(df_pbp[stat_name] == df_pbp['HOME_PLAYER_ID_2'])
                                                  |(df_pbp[stat_name] == df_pbp['HOME_PLAYER_ID_1'])
                                                  , 1, 0)
  else:
    if (if_team_stat == True):
      df_pbp[new_column_name] = np.where((df_pbp[stat_name] == df_pbp['AWAY_PLAYER_ID_5'])
                                                  |(df_pbp[stat_name] == df_pbp['AWAY_PLAYER_ID_4'])
                                                  |(df_pbp[stat_name] == df_pbp['AWAY_PLAYER_ID_3'])
                                                  |(df_pbp[stat_name] == df_pbp['AWAY_PLAYER_ID_2'])
                                                  |(df_pbp[stat_name] == df_pbp['AWAY_PLAYER_ID_1'])
                                                  |(df_pbp[stat_name] == df_pbp['PLAYER1_ID'])
                                                  , 1, 0)
    else:
      df_pbp[new_column_name] = np.where((df_pbp[stat_name] == df_pbp['AWAY_PLAYER_ID_5'])
                                                  |(df_pbp[stat_name] == df_pbp['AWAY_PLAYER_ID_4'])
                                                  |(df_pbp[stat_name] == df_pbp['AWAY_PLAYER_ID_3'])
                                                  |(df_pbp[stat_name] == df_pbp['AWAY_PLAYER_ID_2'])
                                                  |(df_pbp[stat_name] == df_pbp['AWAY_PLAYER_ID_1'])
                                                  , 1, 0)


In [ ]:
df_pbp['prev_home_scored'] = df_pbp['HOME_SCORE'].shift(1)
df_pbp['prev_home_scored'] = df_pbp['prev_home_scored'].fillna(0)

In [ ]:
df_pbp['scored_h'] = df_pbp['HOME_SCORE'] - df_pbp['prev_home_scored']
df_pbp['scored_h'] = np.where(df_pbp['scored_h'] < 0, 0, df_pbp['scored_h'])

In [ ]:
df_pbp['prev_away_scored'] = df_pbp['AWAY_SCORE'].shift(1)
df_pbp['prev_away_scored'] = df_pbp['prev_away_scored'].fillna(0)

In [ ]:
df_pbp['scored_a'] = df_pbp['AWAY_SCORE'] - df_pbp['prev_away_scored']
df_pbp['scored_a'] = np.where(df_pbp['scored_a'] < 0, 0, df_pbp['scored_a'])

In [ ]:
df_pbp['3_pointers_made_h'] = np.where(df_pbp['scored_h'] == 3, 1, 0)
df_pbp['2_pointers_made_h'] = np.where(df_pbp['scored_h'] == 2, 1, 0)
df_pbp['3_pointers_made_a'] = np.where(df_pbp['scored_a'] == 3, 1, 0)
df_pbp['2_pointers_made_a'] = np.where(df_pbp['scored_a'] == 2, 1, 0)
df_pbp['free_throws_made_h'] = np.where(df_pbp['scored_h'] == 1, 1, 0)
df_pbp['free_throws_made_a'] = np.where(df_pbp['scored_a'] == 1, 1, 0)

In [ ]:
statsCounter('REBOUND_PLAYER_ID', 'rebounds_h', True, True)
statsCounter('ASSIST_PLAYER_ID', 'assists_h', False, True)
statsCounter('BLOCK_PLAYER_ID', 'blocks_h', False, True)
statsCounter('FOULED_BY_PLAYER_ID', 'fouls_h', False, True)
statsCounter('STEAL_PLAYER_ID', 'steals_h', False, True)
statsCounter('TURNOVER_PLAYER_ID', 'turnovers_h', True, True)


statsCounter('REBOUND_PLAYER_ID', 'rebounds_a', True, False)
statsCounter('ASSIST_PLAYER_ID', 'assists_a', False, False)
statsCounter('BLOCK_PLAYER_ID', 'blocks_a', False, False)
statsCounter('FOULED_BY_PLAYER_ID', 'fouls_a', False, False)
statsCounter('STEAL_PLAYER_ID', 'steals_a', False, False)
statsCounter('TURNOVER_PLAYER_ID', 'turnovers_a', True, False)


In [ ]:
df_lineups_h = df_pbp[['lineup_h', 'time_per_action', 'scored_h','2_pointers_made_h', '3_pointers_made_h', 'free_throws_made_h','assists_h', 'rebounds_h', 'blocks_h', 'steals_h', 'turnovers_h', 'fouls_h', 'scored_a']]

In [ ]:
df_lineups_a = df_pbp[['lineup_a', 'time_per_action', 'scored_a','2_pointers_made_a', '3_pointers_made_a', 'free_throws_made_a','assists_a', 'rebounds_a', 'blocks_a', 'steals_a', 'turnovers_a', 'fouls_a', 'scored_h']]

In [ ]:
df_lineups_h = df_lineups_h.rename(columns={"lineup_h": "lineup", "time_per_action": "minutes", "scored_h": "points_scored", "2_pointers_made_h": "2_pointers_made", "3_pointers_made_h": "3_pointers_made", "free_throws_made_h": "free_throws_made", "assists_h": "assists", "rebounds_h": "rebounds", "blocks_h": "blocks", "steals_h": "steals", "turnovers_h": "turnovers", "fouls_h": "fouls", "scored_a": "points_allowed"})

In [ ]:
df_lineups_a = df_lineups_a.rename(columns={"lineup_a": "lineup", "time_per_action": "minutes", "scored_a": "points_scored", "2_pointers_made_a": "2_pointers_made", "3_pointers_made_a": "3_pointers_made", "free_throws_made_a": "free_throws_made", "assists_a": "assists", "rebounds_a": "rebounds", "blocks_a": "blocks", "steals_a": "steals", "turnovers_a": "turnovers", "fouls_a": "fouls", "scored_h": "points_allowed"})

In [ ]:
frames = [df_lineups_h, df_lineups_a]
df_results = pd.concat(frames)

In [ ]:
df_lineups_grouped = df_results.groupby(by=["lineup"]).sum()
df_lineups_grouped['minutes'] = round(df_lineups_grouped['minutes'] / 60, 2)
df_lineups_grouped = df_lineups_grouped[df_lineups_grouped.minutes >= 100]

In [ ]:
df_lineups_grouped.to_csv('nba_pbp_2016-2019_grouped.csv', index=True)